# Object Detection (객체탐지)


사전 완료 된 사항 
- 코랩에서 학습 한 뒤 가중치(weights)와 cfg 파일을 내려받음 

---

**Image & Video detection 시 실행 해야하는 사전 코드** 

1.   필요한 Module import 

In [ ]:
import cv2
import numpy as np 

2. 라벨링 객체 선언하기

In [ ]:
# 라벨링 되어있는 폴더의 classes.txt 안에 라벨내용 글씨그대로(대소문자주의)가져와야합니다. 
# 이미지 & 영상 화면에 박싱할 때 해당 라벨명을 사용함 
LABELS = ['Budgerigar',
'Cockatiel',
'hoopoe',
'Corvus',
'Passer',
'Agapornis'
]

3. confidence & threshold 수치 선언 

In [ ]:
# object detection 하려는 사물 정확도 확률에 대한 임계치 정보
# 다른 객체를 자꾸 탐지하게 된다면 confidence 값과 threshold 값을 올려볼 수 있다.(신뢰도를 올리는 방법 ) 

CONFIDENCE = 0.3  # detection 한 사물이 클래스에 해당할 확률
THRESHOLD = 0.3 #NMS(Num Max Suppression) 예측한 bounding 박스 중 정확한것을 선택하는기법 (0~1 사이의값)

4. 코랩에서 학습시킨  cfg ,가중치 파일 불러오기 

In [1]:
net = cv2.dnn.readNetFromDarknet('C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\yolov4_custom.cfg',
                                 'C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\yolov4_custom_last.weights')

## 사전사항 full code 

In [1]:
import cv2
import numpy as np

LABELS = ['Budgerigar',
'Cockatiel',
'hoopoe',
'Corvus',
'Passer',
'Agapornis'
]

CONFIDENCE = 0.3  
THRESHOLD = 0.3 

net = cv2.dnn.readNetFromDarknet('C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\yolov4_custom.cfg',
                                 'C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\yolov4_custom_last.weights')

## Image 

In [20]:
def image_detection():     # 이미지 경로
    
    detection_imag_path = input("1. detection test 할 이미지 경로 입력하세요: \n(주의:경로 역슬러쉬 두번 들어가야 합니다.)\n")
    result_name = input("\n2. detection 한 결과 저장할 이미지명 입력하세요 : ")
    
    img = cv2.imread(detection_imag_path) # 이미지 읽기 (수치화)
    H, W, _ = img.shape 
    
    # 이미지를 리사이즈, 정규화 & 신경망에 흘려보내기 
    blob = cv2.dnn.blobFromImage(img, scalefactor=1/255., size=(416, 416), swapRB=True)
    net.setInput(blob) 
    output = net.forward() 

    boxes, confidences, class_ids = [], [], []
  
    for det in output:         
        box = det[:4]     
        scores = det[5:]        
        class_id = np.argmax(scores) 
        confidence = scores[class_id] 
    
        if confidence > CONFIDENCE:
            cx, cy, w, h = box * np.array([W, H, W, H])
            x = cx - (w / 2)
            y = cy - (h / 2)
            boxes.append([int(x), int(y), int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(class_id)
    

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE, THRESHOLD)
    
    if len(idxs) > 0: # confidence 가 0.3 보다 크면  
        for i in idxs.flatten():
            x, y, w, h = boxes[i] # BOXES 리스트의 정보 (x,y,w,h)
            cv2.rectangle(img, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255,0), thickness=1)
            
            # 박스 위에 추축한 classes text 출력 
            cv2.putText(img, 
                        text='%s %.2f %d' % (LABELS[class_ids[i]], confidences[i], w),
                        org=(x, y -6 ), 
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                        fontScale=0.5, 
                        color=(255, 0, 0), 
                        thickness=2)
    
    # 디텍션한 결과를 저장하기 
    cv2.imwrite('C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\result\\%s.png'%result_name, img)
    print("\nfinish")

In [17]:
# 디텍션 실행 
image_detection()

1. etection test 할 이미지 경로 입력하세요: 
(주의:경로 역슬러쉬 두번 들어가야 합니다.)
C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\test\\test5.jpg

2. detection 한 결과 이미지명 입력하세요 : test5_result

finish


## Video

In [21]:
def video_detection() : 
    # 원본 동영상 위치와 이름 경로 
    detection_video_path = input("1. vedio_detection test 할 비디오 경로 입력하세요: \n(주의:경로 역슬러쉬 두번 들어가야 합니다.)\n")
    result_name = input("\n2. vedio_detection 한 결과 저장할 비디오명 입력하세요 : \n(비디오 저장형식은 mp4 로 저장됩니다.\n)")
    
    cap = cv2.VideoCapture(detection_video_path)

    font = cv2.FONT_HERSHEY_PLAIN # 동영상 디텍션시 나타나는 폰트 
    colors = np.random.uniform(0, 255, size=(100, 3)) # 글씨 색상 

    # 동영상 프레임 안에 가로, 세로 사이즈 변수( frame_size) 담기
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 동영상형식 : mp4  
    fps = cap.get(cv2.CAP_PROP_FPS) # 동영상 원본 (cap)을 초당 프레임단위로 가져오기

    # 결과 동영상의 저장될 최종 위치 
    # 오브젝트 디텍션 될 detection.mp4 를 생성하기 위한 out 이라는 이름 변수 생성 
    out = cv2.VideoWriter('C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\result\\%s.mp4'%result_name, 
                          fourcc,  # 저장형식
                          fps,     # 
                          (int(frame_size[0]), 
                           int(frame_size[1])))

    # 무한 loop 돌림 
    while True:
         # 동영상에서 frame 을 가져와 img 에 담기 ret 은 True 할당 
            #(가져오기만하면 나중에 루프 끝나기 위한 값 )
        ret, img = cap.read()
        print(img)
        if not ret: # True 가 끝나면 (=Flase이면) 루프문 종료 
            break

        # 동영상의 frame 에 대해서 넓이와 높이 구해야 하므로 cv2.CAP_PROP_FRAME_WIDTH 와 cv2.CAP_PROP_FRAME_HEIGHT 를 이용 
        width,height = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # 정규화 (학습데이터와 맞춤) RGB 로 학습했음으로 swapRB = True(BGR 변경)
        blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)

        # 다크넷(net)신경망에 전처리된 blob frame 을 넣기 (setinput)  
        net.setInput(blob)

        # 다크넷 신경망에서 맨 마지막층들의 이름가져오기 output_layers_names   
        output_layers_names = net.getUnconnectedOutLayersNames()

        # forward 신경망 
        layerOutputs = net.forward(output_layers_names)

        # 사물 검출을시 사용할 비어있는 리스트 3개 선언 
        boxes,confidences,class_ids = [],[],[]

        for output in layerOutputs: # 507 개의 박스데이터 불러오기 
            # 밑의 4줄의 코드 : 박스의 원본사이즈를 구하는 코드 
            for detection in output: # list 안의list 라 이중 loop 문 구성 
                scores = detection[5:] # 507 개의 박스에서 각 클래스의 확률 담음 
                class_id = np.argmax(scores) # 가장 큰 확률의 인덱스 class_id 담기
                confidence = scores[class_id] # 가장 추출된 인덱스의 의 확률값을 confidence 에 담기

                if confidence > CONFIDENCE: # confidece 의 값이 CONFIDENCE(위에 값 줌) 보다 크다면 (사물이 있을 확률 박스를 담아내기 위한 코드)
                    center_x = int(detection[0]*width) # 0~ 1 사이로 변경된 좌표값을 실제 좌표로 변경 
                    center_y = int(detection[1]*height)
                    w = int(detection[2]*width) # 실제 가로 
                    h = int(detection[3]*height)# 실제 높이 로 변경 

                    x = int(center_x - w/2) # 박스의 중심 x 좌표 
                    y = int(center_y - h/2) # 박스의 중심 y 좌표 

                    boxes.append([x, y, w, h]) # 원하는 사물이 있는 박스정보를 boxes 에 넣기 
                    confidences.append((float(confidence))) # 0.2 보다 큰 확률 데이터를 confidece 에 넣기
                    class_ids.append(class_id) # class_id 에는 0 또는 1 의 값 (고라니, 너구리)

        # 0.2 는 사물이 맞을 확률에 대한 임계치 ( 값은 내가 지정하는것 )
        # 0.4 는 박스 안에 사물이 있을 확률에 대한 임계치 지정 
        # 해당 임계치보다 더 큰 박스 정보들만 indexes 에 담기 
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE, THRESHOLD)

        if len(indexes)>0: # indexes 에 값이 존재하면
            for i in indexes.flatten(): # boxes 리스트에 (x,y,w,h) 에 대한 정보를 
                x, y, w, h = boxes[i]   # x ,y,w,h 에 넣습니다. 
                label = str(LABELS[class_ids[i]])# 클래스 label 넣고 
                confidence = str(round(confidences[i],2)) # 각클래스일 확률을 confidece 에 넣고 
                color = colors[i] # i 의 값에 따라 색상이 달라짐 
                cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # 박싱해주는 과정 
                # 박스를 그리는데 pt1 에 중심좌표를 두고 pt2 에는 
                #중심좌표로부터의 넓이와 높이 정보를 주고 선두께를 2로 하여 그린다. 

                # cv2.putText 를 이용하여 라벨과 확률을 중심좌표에서 y 축으로 +20 지점에 표시한다
                cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)

        # 동영상 frame 마다 표시되는 속도로 play 하기 
        cv2.imshow('Image', img) # img 가 프레임 (속도가 좀 느림 )

        # 동영상을 아까 저장할 경로와 지정해놓은 out 객체에 프레임을 write(기록) 한다. 
        out.write(img)

        # cv2.waitKey(1) 은 입력을 기다리는 대기 함수 
        key = cv2.waitKey(1)
        if key==27: # 취소키를 누르면 (27)
            break   # 멈춘다. 


    out.release()    # 디텍션된 동영상에 대한 데이터가 있는 변수 지움     
    cv2.destroyAllWindows() # 열어 놓은 윈도우(프레임당 움직이고있는)창 닫기
    
    print("detection finish")

In [22]:
video_detection()

1. vedio_detection test 할 이미지 경로 입력하세요: 
(주의:경로 역슬러쉬 두번 들어가야 합니다.)
C:\\deeplearning\\[Portfolio]\\Birds_ObDetection\\test\\test_video1.mp4

2. vedio_detection 한 결과 저장할 비디오명 입력하세요 : 
(비디오 저장형식은 mp4 로 저장됩니다.
)test_video1_result
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


...

[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [218 216 221]
  [218 216 221]
  [218 216 221]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [218 216 221]
  [220 214 222]
  [220 214 222]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [218 216 221]
  [220 214 222]
  [220 214 222]]]
[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ..

[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [220 214 222]
  [220 214 222]
  [220 214 222]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [220 214 222]
  [220 214 222]
  [220 214 222]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [220 214 222]
  [220 214 222]
  [220 214 222]]]
[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ..

[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]]
[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ..

[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]]
[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ..

[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [217 217 217]
  [217 217 217]
  [217 217 217]]]
[[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [181 195 200]
  [181 195 200]
  [181 195 200]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ..

## 실시간 웹캠 detection

- 실시간 웹캠 안에 신경망 내용(비디오 디텍션 부분) 이 들어가게 됩니다. 

- 사전사항 반드시 실행 되어있어야합니다. 

- 모듈 임포트 

In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

### Stop button
- 버튼 생성 : 사진 촬영되는것 stop 시키기 위함 

In [3]:
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',# 버튼의 글씨 
    disabled=False,  # 버튼 활성화 (Ture) 비활성화 (False)
    button_style='info', # 'success', 'info', 'warning', 'danger' or '' # style
    tooltip='Description', # 버튼에 대한 설명
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

- 폰트 , 컬러

In [4]:
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))

### Display function 
- 실시간 웹캠 실행하는 함수 

In [5]:
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    
    while True:
        _, frame = cap.read()       # 캠 카메라에서 보내는 frame
        frame2 = cv2.flip(frame, 1) # if your camera reverses your image
              
        if stopButton.value==True:      # stop 버튼 눌렀을 때 작동되는 코드
            cap.release()
            display_handle.update(None)
###############################################################################
        
        ################################# 신경망내용 ###########################################
        # 영상의 넓이와 높이를 받아서 width, height에 각각 넣음    
        width,height = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
        #print(type(frame)) - 영상 데이터 전처리 하는 부분
        blob = cv2.dnn.blobFromImage(frame2, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
        net.setInput(blob)
        output_layers_names = net.getUnconnectedOutLayersNames()
        layerOutputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layerOutputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > CONFIDENCE:
                    center_x = int(detection[0]*width)
                    center_y = int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    x = int(center_x - w/2)
                    y = int(center_y - h/2)

                    boxes.append([x, y, w, h])
                    confidences.append((float(confidence)))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE, THRESHOLD) # 자꾸 사람보고 개라고 해서 신뢰도 올림

        if len(indexes)>0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(LABELS[class_ids[i]])
                confidence = str(round(confidences[i],2))
                color = colors[i]
                cv2.rectangle(frame2, (x,y), (x+w, y+h), color, 2)
                cv2.putText(frame2, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)
        
        # numpy array 형식의 frame을 리스트로 변환
        _, frame3 = cv2.imencode('.jpeg', frame2)
        
        # 주피터 노트북에서 실시간으로 영상을 display 함
        display_handle.update(Image(data=frame3.tobytes()))

        # 취소 키를 누르면 while loop문이 종료됨
        key = cv2.waitKey(1)
        if key==27:
            break

        
    out.release()        
    cv2.destroyAllWindows()

        #####################

### 실행하기(run) 

In [6]:
# 병렬로 실행하기 
display(stopButton) # stopbutton 먼저 보여준 뒤 
thread = threading.Thread(target=view, args=(stopButton,)) # 입력파라미터는 : stopbutton 
thread.start()

# 참고 사이트 
#https://abauville.medium.com/display-your-live-webcam-feed-in-a-jupyter-notebook-using-opencv-d01eb75921d1

ToggleButton(value=False, button_style='info', description='Stop', icon='square', tooltip='Description')

None

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\hyunjeong\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\hyunjeong\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\HYUNJE~1\AppData\Local\Temp/ipykernel_30672/2831101872.py", line 20, in view
cv2.error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

